# Predictions: Heart Diseases

Prediction of heart disease given medical informartion. 

## Libraries

In [ ]:
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

pd.set_option('display.max_colwidth', None)

In [ ]:
MAX_DATASET_SAMPLES = 10000 # Datasets will the capped to this sample_size to keep experiments fast 

## Import Data

In [3]:
datasets = [
    {
        "name": "Heart Attack Prediction Dataset",
        "path": "data/heart.csv",
        "features_numerical": ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak'],
        "features_categorical": ['sex', 'cp', 'fbs', 'restecg', 'exng', 'slp', 'caa', 'thall'],
        "target": "output",
    },

    {
        "name": "Heart Failure Prediction",
        "path": "data/heart(2).csv",
        "features_numerical": ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak' ],
        "features_categorical": ['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope'],
        "target": "HeartDisease",
    },

    {
        "name": "Heart Failure Prediction 2",
        "path": "data/heart_failure_clinical_records_dataset.csv",
        "features_numerical": ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time' ],
        "features_categorical": ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking',],
        "target": "DEATH_EVENT",
    },

    {
        "name": "Cardiovascular Disease Dataset",
        "path": "data/cardio_train.csv",
        "features_numerical": ['age', 'height', 'weight', 'ap_hi', 'ap_lo' ],
        "features_categorical": ['gender', 'cholesterol', 'gluc', 'smoke', 'alco', 'active'],
        "target": "cardio",
    },

    {
        "name": "Indicators of Heart Disease",
        "path": "data/heart_2020_cleaned.csv",
        "features_numerical": ['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime',  ],
        "features_categorical": ['Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Sex', 'AgeCategory', 'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'Asthma', 'KidneyDisease', 'SkinCancer', ],
        "target": "HeartDisease",
    },

]


In [4]:
dataset_path = datasets[0]["path"]
df = pd.read_csv(dataset_path)
df

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


## EDA

In [5]:
def describe_dataframe(df:pd.DataFrame)->pd.DataFrame:
    df_describe = df.describe()
    df_describe.loc['dtype'] = df_describe.dtypes
    df_describe.loc['null_count'] = df_describe.isnull().sum()
    print("df.shape:", df.shape)
    return df_describe

describe_dataframe(df)

df.shape: (303, 14)


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,303.0,303.0,303.0,303.0,303.0,303.0,303.0,303.0,303.0,303.0,303.0,303.0,303.0,303.0
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.39934,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.52586,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.0,0.0,0.0,94.0,126.0,0.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,47.5,0.0,0.0,120.0,211.0,0.0,0.0,133.5,0.0,0.0,1.0,0.0,2.0,0.0
50%,55.0,1.0,1.0,130.0,240.0,0.0,1.0,153.0,0.0,0.8,1.0,0.0,2.0,1.0
75%,61.0,1.0,2.0,140.0,274.5,0.0,1.0,166.0,1.0,1.6,2.0,1.0,3.0,1.0
max,77.0,1.0,3.0,200.0,564.0,1.0,2.0,202.0,1.0,6.2,2.0,4.0,3.0,1.0
dtype,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
null_count,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Define Model

In [6]:
# Suppress onehot transformation warnings
import warnings
import re
original_showwarning = warnings.showwarning
def custom_showwarning(message, category, filename, lineno, file=None, line=None):
    if re.match(r"Found unknown categories in columns \[\d+\] during transform\. These unknown categories will be encoded as all zeros", str(message)):
        return  # Suppress the specific warning
    original_showwarning(message, category, filename, lineno, file, line)
warnings.showwarning = custom_showwarning

In [7]:
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression

def create_pipeline(dataset:dict, classifier:BaseEstimator)->Pipeline:

    features_numerical = dataset["features_numerical"]
    features_categorical = dataset["features_categorical"]
    target = dataset['target']
    features_all = features_numerical+features_categorical

    numerical_pipeline = Pipeline(steps=[
        # ('impute', SimpleImputer(strategy='mean')),
        ('scale', StandardScaler())
    ])
    categorical_pipeline = Pipeline(steps=[
        # ('impute', SimpleImputer(strategy='most_frequent')),
        ('one_hot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
    ])
    column_transformer = ColumnTransformer(transformers=[
        ('numerical', numerical_pipeline, features_numerical),
        ('categorical', categorical_pipeline, features_categorical)
    ])
    pipeline = Pipeline(steps=[
        ('preprocessor', column_transformer),
        ('classifier', classifier)
    ])
    
    return pipeline

# pipeline example
create_pipeline(datasets[0], LogisticRegression())

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scale',
                                                                   StandardScaler())]),
                                                  ['age', 'trtbps', 'chol',
                                                   'thalachh', 'oldpeak']),
                                                 ('categorical',
                                                  Pipeline(steps=[('one_hot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['sex', 'cp', 'fbs',
                                                   'restecg', 'exng', 'slp',
                                                   'caa', 'thall'])])),
                ('classifier', LogisticRegression())])

In [8]:
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier


classifiers = [
    LogisticRegression(n_jobs=-1),
    MLPClassifier(max_iter=1000, alpha=3),
    RandomForestClassifier(n_jobs=-1),
    ExtraTreesClassifier(n_jobs=-1),
    # paper
    StackingClassifier(n_jobs=-1, estimators=[('lr', LogisticRegression(n_jobs=-1)),    ('rf', RandomForestClassifier(n_jobs=-1))], final_estimator=LogisticRegression()),
    StackingClassifier(n_jobs=-1, estimators=[('svm', SVC(probability=True)),           ('mlp', MLPClassifier(max_iter=1000, alpha=3))], final_estimator=LogisticRegression()),
    StackingClassifier(n_jobs=-1, estimators=[('lr', LogisticRegression(n_jobs=-1)),    ('mlp', MLPClassifier(max_iter=1000, alpha=3))], final_estimator=LogisticRegression()),
    # Arpitha
    StackingClassifier(n_jobs=-1, estimators=[('knn', KNeighborsClassifier(n_neighbors=5)), ('gb', GradientBoostingClassifier())], final_estimator=LogisticRegression()),

]

## Training

In [9]:
results = {}

for dataset in datasets:
    print(40*"##")
    print("Dataset:", dataset["name"])
    dataset_name = dataset["name"]
    results[dataset_name] = {}
    # Load Data
    df = pd.read_csv(dataset['path'])
    if len(df)>MAX_DATASET_SAMPLES: # Use subsample if the data is too large
        df = df.sample(n=MAX_DATASET_SAMPLES, replace=False, random_state=42)

    for classifier in classifiers:
        classifier_name = str(classifier)
        results[dataset_name][classifier_name] = {}
        
        # Crossvalidation
        accuracies = []
        f1_scores = []
        auc_scores = []
        for i in range(10):
            features_numerical = dataset["features_numerical"]
            features_categorical = dataset["features_categorical"]
            target = dataset['target']
            features_all = features_numerical+features_categorical

            # Data Split
            df_train, df_val = train_test_split(df, test_size=0.2)
            X_train = df_train[features_all]
            Y_train = df_train[target]
            X_val = df_val[features_all]
            Y_val = df_val[target]

            # Create pipeline
            pipeline = create_pipeline(dataset, classifier)
            
            # Train and predict
            pipeline.fit(X_train, Y_train)
            y_pred_val = pipeline.predict(X_val)
            y_prob_val = pipeline.predict_proba(X_val)[:, 1]

            # Metrics
            accuracies.append(accuracy_score(Y_val, y_pred_val))
            auc_scores.append(roc_auc_score(Y_val, y_prob_val))
            if (Y_val == 'Yes').any():
                f1_scores.append(f1_score(Y_val, y_pred_val, pos_label='Yes'))
            else:
                f1_scores.append(f1_score(Y_val, y_pred_val))

        accuracy = np.average(accuracies)
        results[dataset_name][classifier_name]["accuracy"] = accuracy
        print(classifier, "average acc:", accuracy)

        f1 = np.average(f1_scores)
        results[dataset_name][classifier_name]["f1"] = f1
        print(classifier, "average f1:", f1)

        auc = np.average(auc_scores)
        results[dataset_name][classifier_name]["auc"] = auc
        print(classifier, "average auc:", auc)

# pprint(results)

################################################################################
Dataset: Heart Attack Prediction Dataset
LogisticRegression(n_jobs=-1) average acc: 0.8704918032786886
LogisticRegression(n_jobs=-1) average f1: 0.8823219005438674
LogisticRegression(n_jobs=-1) average auc: 0.9298045596116982
MLPClassifier(alpha=3, max_iter=1000) average acc: 0.8229508196721312
MLPClassifier(alpha=3, max_iter=1000) average f1: 0.8393326492494347
MLPClassifier(alpha=3, max_iter=1000) average auc: 0.8945463091174665
RandomForestClassifier(n_jobs=-1) average acc: 0.8344262295081968
RandomForestClassifier(n_jobs=-1) average f1: 0.8456205400680249
RandomForestClassifier(n_jobs=-1) average auc: 0.9007784386623097
ExtraTreesClassifier(n_jobs=-1) average acc: 0.8442622950819672
ExtraTreesClassifier(n_jobs=-1) average f1: 0.8574007160828756
ExtraTreesClassifier(n_jobs=-1) average auc: 0.9147061043986795
StackingClassifier(estimators=[('lr', LogisticRegression(n_jobs=-1)),
                          

In [10]:
def format_results(results_dict:dict)->pd.DataFrame:
    """
    NOTE: Docstring created with ChatGPT, edited by a human
    Converts a nested dictionary of results into a DataFrame with Multi-Index columns.
    
    Parameters:
    - results_dict (dict): Nested dictionary where the first level keys are datasets, 
      the second level keys are models, and the leaf values are dictionaries of metrics.
      
    Returns:
    - pd.DataFrame: A DataFrame with Multi-Index columns for datasets and models, 
      and rows for metrics.
    """
    results_df = pd.DataFrame.from_dict(
        {(dataset, model): metrics for dataset, model_dict in results_dict.items() for model, metrics in model_dict.items()},
        orient="index",
    )

    # Set MultiIndex for the columns
    results_df.index = pd.MultiIndex.from_tuples(results_df.index, names=["Dataset", "Model"])
    results_df.columns.name = "Metric"

    return results_df

display(format_results(results))


Metric                                                                              accuracy  \
Dataset                         Model                                                          
Heart Attack Prediction Dataset LogisticRegression(n_jobs=-1)                       0.870492   
                                MLPClassifier(alpha=3, max_iter=1000)               0.822951   
                                RandomForestClassifier(n_jobs=-1)                   0.834426   
                                ExtraTreesClassifier(n_jobs=-1)                     0.844262   
                                StackingClassifier(estimators=[('lr', LogisticR...  0.860656   
                                StackingClassifier(estimators=[('svm', SVC(prob...  0.824590   
                                StackingClassifier(estimators=[('lr', LogisticR...  0.840984   
                                StackingClassifier(estimators=[('knn', KNeighbo...  0.790164   
Heart Failure Prediction        LogisticRegression(n_jobs=-1)                       0.865217   
                                MLPClassifier(alpha=3, max_iter=1000)               0.878804   
                                RandomForestClassifier(n_jobs=-1)                   0.877717   
                                ExtraTreesClassifier(n_jobs=-1)                     0.863043   
                                StackingClassifier(estimators=[('lr', LogisticR...  0.882065   
                                StackingClassifier(estimators=[('svm', SVC(prob...  0.864130   
                                StackingClassifier(estimators=[('lr', LogisticR...  0.867935   
                                StackingClassifier(estimators=[('knn', KNeighbo...  0.846739   
Heart Failure Prediction 2      LogisticRegression(n_jobs=-1)                       0.828333   
                                MLPClassifier(alpha=3, max_iter=1000)               0.818333   
                                RandomForestClassifier(n_jobs=-1)                   0.836667   
                                ExtraTreesClassifier(n_jobs=-1)                     0.808333   
                                StackingClassifier(estimators=[('lr', LogisticR...  0.811667   
                                StackingClassifier(estimators=[('svm', SVC(prob...  0.843333   
                                StackingClassifier(estimators=[('lr', LogisticR...  0.828333   
                                StackingClassifier(estimators=[('knn', KNeighbo...  0.858333   
Cardiovascular Disease Dataset  LogisticRegression(n_jobs=-1)                       0.718550   
                                MLPClassifier(alpha=3, max_iter=1000)               0.683150   
                                RandomForestClassifier(n_jobs=-1)                   0.716300   
                                ExtraTreesClassifier(n_jobs=-1)                     0.709200   
                                StackingClassifier(estimators=[('lr', LogisticR...  0.731550   
                                StackingClassifier(estimators=[('svm', SVC(prob...  0.724200   
                                StackingClassifier(estimators=[('lr', LogisticR...  0.728800   
                                StackingClassifier(estimators=[('knn', KNeighbo...  0.740350   
Indicators of Heart Disease     LogisticRegression(n_jobs=-1)                       0.906200   
                                MLPClassifier(alpha=3, max_iter=1000)               0.911000   
                                RandomForestClassifier(n_jobs=-1)                   0.904750   
                                ExtraTreesClassifier(n_jobs=-1)                     0.889200   
                                StackingClassifier(estimators=[('lr', LogisticR...  0.907450   
                                StackingClassifier(estimators=[('svm', SVC(prob...  0.905650   
                                StackingClassifier(estimators=[('lr', LogisticR...  0.905550   
                                StackingClassifier(estimators=[(

In [16]:
def average_metrics(results):
    # Initialize a dictionary to store metrics by model
    averaged_results = {}
    
    # Iterate through datasets and collect metrics for each model
    for models in results.values():
        for model, metrics in models.items():
            if model not in averaged_results:
                averaged_results[model] = {'accuracy': [], 'f1': [], 'auc': []}
            for metric, value in metrics.items():
                averaged_results[model][metric].append(value)
    
    # Calculate averages for each model
    for model, metrics in averaged_results.items():
        averaged_results[model] = {metric: np.mean(values) for metric, values in metrics.items()}
    
    # Convert the averaged results into a DataFrame
    df = pd.DataFrame.from_dict(averaged_results, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Model'}, inplace=True)
    
    return df

average_metrics(results)

,Model,accuracy,f1,auc
0,LogisticRegression(n_jobs=-1),0.837759,0.668831,0.865197
1,"MLPClassifier(alpha=3, max_iter=1000)",0.822848,0.622239,0.850318
2,RandomForestClassifier(n_jobs=-1),0.833972,0.657048,0.859549
3,ExtraTreesClassifier(n_jobs=-1),0.822808,0.653343,0.852895
4,"StackingClassifier(estimators=[('lr', LogisticRegression(n_jobs=-1)),\n ('rf', RandomForestClassifier(n_jobs=-1))],\n final_estimator=LogisticRegression(), n_jobs=-1)",0.838678,0.676637,0.873518
5,"StackingClassifier(estimators=[('svm', SVC(probability=True)),\n ('mlp', MLPClassifier(alpha=3, max_iter=1000))],\n final_estimator=LogisticRegression(), n_jobs=-1)",0.832381,0.658294,0.865627
6,"StackingClassifier(estimators=[('lr', LogisticRegression(n_jobs=-1)),\n ('mlp', MLPClassifier(alpha=3, max_iter=1000))],\n final_estimator=LogisticRegression(), n_jobs=-1)",0.834320,0.667214,0.870074
7,"StackingClassifier(estimators=[('knn', KNeighborsClassifier()),\n ('gb', GradientBoostingClassifier())],\n final_estimator=LogisticRegression(), n_jobs=-1)",0.828717,0.662529,0.863444
